In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/segmentation/segmentation_models/efficientnet_pytorch-0.6.3.xyz /tmp/pip/cache/efficientnet_pytorch-0.6.3.tar.gz
!cp ../input/segmentation/segmentation_models/pretrainedmodels-0.7.4.xyz /tmp/pip/cache/pretrainedmodels-0.7.4.tar.gz
!cp ../input/segmentation/segmentation_models/segmentation-models-pytorch-0.1.2.xyz /tmp/pip/cache/segmentation_models_pytorch-0.1.2.tar.gz
!cp ../input/segmentation/segmentation_models/timm-0.1.20-py3-none-any.whl /tmp/pip/cache/
!cp ../input/segmentation/segmentation_models/timm-0.2.1-py3-none-any.whl /tmp/pip/cache/
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet-pytorch
!pip install --no-index --find-links /tmp/pip/cache/ segmentation-models-pytorch

In [ ]:
import segmentation_models_pytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import cv2
import os
import gc
from tqdm.notebook import tqdm
import rasterio
from rasterio.windows import Window

# from fastai.vision.all import *
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")
import torch

In [ ]:
sz = 512   #the size of tiles
reduce = 2 #reduce the original images by 4 times
TH = 0.4  #threshold for positive predictions
DATA = '../input/hubmap-kidney-segmentation/test/'
# MODELS = [f'../input/hubmap-fast-ai-starter/model_{i}.pth' for i in range(4)]
df_sample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
# bs = 64
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
overlap = 230
DEVICE

In [ ]:
mean = np.array([0.65459856,0.48386562,0.69428385])
std = np.array([0.15167958,0.23584107,0.13146145])

identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

In [ ]:
import numba
subm = {}
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

# used for converting the decoded image to rle mask
def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
def rle_decode(mask_rle, shape=(256, 256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')
@numba.njit()
def rle_numba(pixels):
    size = len(pixels)
    points = []
    if pixels[0] == 1: points.append(0)
    flag = True
    for i in range(1, size):
        if pixels[i] != pixels[i-1]:
            if flag:
                points.append(i+1)
                flag = False
            else:
                points.append(i+1 - points[-1])
                flag = True
    if pixels[-1] == 1: points.append(size-points[-1]+1)    
    return points
def rle_numba_encode(image):
    pixels = image.flatten(order = 'F')
    points = rle_numba(pixels)
    return ' '.join(str(x) for x in points)

In [ ]:
def model_pred(models, image, tta=None):
    py = None
    for model in models:
        score = model(image)[0][0]
        if py == None:
            py = score
        else:
            py += score
    py = py/len(models)
    return py

# def model_pred(models, image, tta=None):
#     score = models(image)[0][0]
#     return score

In [ ]:
model = [1,2,3]
[model[0],model[1]]

In [ ]:
import pathlib
WINDOW = 1024

DATA_PATH = '../input/hubmap-kidney-segmentation/'

p = pathlib.Path(DATA_PATH)
numk = 0
for i, filename in enumerate(p.glob('test/*.tiff')):
    file = rasterio.open(filename.as_posix())
    slices = make_grid(file.shape, window=WINDOW, min_overlap=overlap) # this version
    preds = np.zeros(file.shape, dtype=np.float16)
    print('staring predict test data:',filename)
    print(file.shape)
    print(slices)
    models = []
    model = torch.load(f'../input/b7-models/best_0_7.pth')
    models.append(model)
    del model; gc.collect()
    model = torch.load(f'../input/b7-models/best_4.pth')
    models.append(model)
    del model; gc.collect()
    model = torch.load(f'../input/b7-models/best_3.pth')
    models.append(model)
    del model; gc.collect()
    
    
    if file.count != 3:
        print('Image file with subdatasets as channels')
        layers = [rasterio.open(subd) for subd in file.subdatasets]
    for (x1,x2,y1,y2) in slices:
        if file.count == 3: # normal
            image = file.read([1,2,3],
                        window=Window.from_slices((x1,x2),(y1,y2)))
            image = np.moveaxis(image, 0, -1)
            
        else:
            image = np.zeros((WINDOW, WINDOW, 3), dtype=np.uint8)
            for fl in range(3):
                image[:,:,fl] = layers[fl].read(window=Window.from_slices((x1,x2),(y1,y2)))
        image = cv2.resize(image,(image.shape[1]//reduce,image.shape[0]//reduce),
                           interpolation = cv2.INTER_AREA)
        image = img2tensor((image/255 - mean)/std)

        with torch.no_grad():
            image = image.to(DEVICE)
#             print(image.shape)
            score = model_pred([models[0]], image[None,...])
            score2 = model_pred([models[1]], torch.flip(image[None,...], [0, 3]))
            score2 = torch.flip(score2[None,None,...], [3, 0])[0][0]
            score3 = model_pred([models[2]], torch.flip(image[None,...], [1, 2]))
            score3 = torch.flip(score3[None,None,...], [2, 1])[0][0]
            score = (score + score2 + score3)/3
            score = score.cpu()
            score = cv2.resize(np.float32(score), (1024, 1024))
        
            preds[x1:x2,y1:y2] = score*score/(preds[x1:x2,y1:y2] + score) + preds[x1:x2,y1:y2]*preds[x1:x2,y1:y2]/(preds[x1:x2,y1:y2] + score)
            
    del score,score2,score3,image
    del slices,models; gc.collect()
    # -------------------------------------- other overlaps --------------------------------------------------------
    slices = make_grid(file.shape, window=WINDOW, min_overlap=32) # this version
    
    models = []
    model = torch.load(f'../input/0924model/best_0_924.pth')
    models.append(model)
    del model; gc.collect()
    model = torch.load(f'../input/b5testexternal/b5 -test -external/best_0(5).pth')
    models.append(model)
    del model; gc.collect()
    model = torch.load(f'../input/b5testexternal/b5 -test -external/best_4(1).pth')
    models.append(model)
    del model; gc.collect()
    model = torch.load(f'../input/b5testexternal/b5 -test -external/best_3(1).pth')
    models.append(model)
    del model; gc.collect()
    model = torch.load(f'../input/b5testexternal/b5 -test -external/best_1(1).pth')
    models.append(model)
    del model; gc.collect()
    
    
    if file.count != 3:
        print('Image file with subdatasets as channels')
        layers = [rasterio.open(subd) for subd in file.subdatasets]
    for (x1,x2,y1,y2) in slices:
        if file.count == 3: # normal
            image = file.read([1,2,3],
                        window=Window.from_slices((x1,x2),(y1,y2)))
            image = np.moveaxis(image, 0, -1)
            
        else:
            image = np.zeros((WINDOW, WINDOW, 3), dtype=np.uint8)
            for fl in range(3):
                image[:,:,fl] = layers[fl].read(window=Window.from_slices((x1,x2),(y1,y2)))
        image = cv2.resize(image,(image.shape[1]//reduce,image.shape[0]//reduce),
                           interpolation = cv2.INTER_AREA)
        image = img2tensor((image/255 - mean)/std)

        with torch.no_grad():
            image = image.to(DEVICE)
#             print(image.shape)
            score = model_pred([models[0]], image[None,...])
            score2 = model_pred([models[1], models[2]], torch.flip(image[None,...], [0, 3]))
            score2 = torch.flip(score2[None,None,...], [3, 0])[0][0]
            score3 = model_pred([models[3], models[4]], torch.flip(image[None,...], [1, 2]))
            score3 = torch.flip(score3[None,None,...], [2, 1])[0][0]
            score = (score+score2+score3)/3
            
            score = score.cpu()
            score = cv2.resize(np.float32(score), (1024, 1024))
            
            preds[x1:x2,y1:y2] = (score + preds[x1:x2,y1:y2])/2
    
    preds = (preds > TH).astype(np.uint8)    
    subm[i] = {'id':filename.stem, 'predicted': rle_numba_encode(preds)}
    
    del preds; gc.collect()

In [ ]:
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv(f'submission.csv', index=False)
submission
%reset -f out